# Notebook Setup

These code snippets install the appropriate libraries, setup logging for the Python code run in this notebook, and optionally, this can be configured to proxy the requests through a proxy such as a corporate proxy or local proxy (Fiddler or HTTP Toolkit) for local debugging

In [ ]:
%pip install -r requirements.txt

In [ ]:
# Setup logging in debug mode
import logging
import sys

"""This function configures logging for code being run based on the specified level"""
def configure_logging(level="ERROR"):
    try:
        # Convert the level string to uppercase so it matches what the logging library expects
        logging_level = getattr(logging, level.upper(), None)

        # Setup a logging format
        logging.basicConfig(
            level=logging_level,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[logging.StreamHandler(sys.stdout)]
        )
    except Exception as e:
        print(f"Failed to set up logging: {e}", file=sys.stderr)
        sys.exit(1) 

configure_logging("ERROR")

## OPTIONAL

In [ ]:
import os

# Setup proxy through local proxy; bypass proxy for metadata endpoint to avoid any issues with Microsoft authentication libraries
os.environ["HTTP_PROXY"] = "http://127.0.0.1:8000"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:8000"
os.environ["NO_PROXY"] = "169.254.169.254"

# Add private CA root cert to trusted cert bundle for SDKs that use requests and httpx
os.environ["REQUESTS_CA_BUNDLE"] = "/Users/someuser/Library/Preferences/httptoolkit/ca.pem"
os.environ["SSL_CERT_FILE"] = "/Users/someuser/Library/Preferences/httptoolkit/ca.pem"


# Helper functions

In [ ]:
"""This function decodes a JWT token into its header and payload components"""
def decode_jwt(token):
    # Split the token into its three parts
    parts = token.split('.')
    
    # Decode header and payload (add padding if needed)
    def decode_part(part):
        # Add padding if necessary (base64url requires padding to be a multiple of 4)
        padding = 4 - len(part) % 4
        if padding != 4:
            part += '=' * padding
        # Use base64url decoding (replaces - with + and _ with /)
        decoded = base64.urlsafe_b64decode(part)
        return json.loads(decoded)
    
    header = decode_part(parts[0])
    payload = decode_part(parts[1])
    
    return header, payload

# Required functions

## Obtain an access token for the user for the Microsoft Graph API

**Identity Context:** Notebook User

**Credential:** Notebook User's Entra ID credentials

This code in this cell obtains an access token for the Microsoft Graph API for the user that will be running the notebook. This token will be used throughout the notebook and is stored in a variable named user_token. The user must log into Azure CLI prior to running this cell.


In [ ]:
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env', override=True)

# Get a token for Microsoft Graph with the logged in user
credential = DefaultAzureCredential()
scopes = ["https://graph.microsoft.com/.default"]

user_token = credential.get_token(*scopes)

# Create Entra ID app registration for Blueprint Creator App
User identities cannot directly create Entra ID Agent Identity Blueprints and instead these must be creatd by an application through an on-behalf-of or client credentials flow. The user account that creates resources needs to have [appropriate permissions](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/create-blueprint?tabs=microsoft-graph-api).

The user that you use in this notebook must have Global Administrator or a combination of Application Administrator and Agent ID Administrator.

## 1. Create app registration for Blueprint Creator App

**Identity Context:** Notebook User

**Credential:** Notebook User's Access Token for Microsoft Graph

This code in this cell creates an Entra ID application that is configured as a public client with support for the device code flow. Users cannot directly create Entra ID Identity Blueprints and instead must delegate their permissions to an application to perform the creation. 

Once created record the following values in the .env file:
1. appId property should be written to the ENTRA_ID_BLUEPRINT_CREATOR_APP_CLIENT_ID
2. id property should be written to the ENTRA_ID_BLUEPRINT_CREATOR_APP_OBJECT_ID

An application registration (or application object) is an object in Entra ID that defines an application's properties such as its identity, permissions the application defines for itself, the permissions it requires in the user's tenant. There is always one application registration for an app. In multi-tenant scenarios, that application registration exists in the source tenant.



In [ ]:
import requests
import json

"""This function calls the Microsoft Graph API to search for an application registration by its display name"""
def get_app_registration_by_display_name(display_name: str):
    response = requests.get(
        f'https://graph.microsoft.com/v1.0/applications?$filter=displayName eq \'{display_name}\'',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        }
    )
    if response.status_code != 200:
        print(f"Failed to get application registrations: {response.status_code} - {response.text}")
        return None
    # Parse the response and get the value array and return an empty list if not found
    apps = response.json().get('value', [])
    return apps[0] if apps else None

"""This function calls the Microsoft Graph API to create an app registration"""
def create_app_registration(display_name: str):
    existing_app_registration = get_app_registration_by_display_name(display_name)

    # Before creating the app registration, check if it already exists
    if existing_app_registration is not None:
        print(f"Application registration with display name '{display_name}' already exists with id: {existing_app_registration['id']}")
        return existing_app_registration
    body = {
        "displayName": display_name,
        "signInAudience": "AzureADMyOrg",
        # Allow public client flows for the purposes of this demo
        "isFallbackPublicClient": True
    }
    response = requests.post(
        'https://graph.microsoft.com/v1.0/applications',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        },
        json=body
    )
    if response.status_code != 201:
        print(f"Failed to create application registration: {response.status_code} - {response.text}")
        return None
    return response.json()

# Set the display name to be used for the application registration
app_display_name = "Entra ID Agent Identity Blueprint Creator Demo 2"

# Create the application registration
app_registration = create_app_registration(app_display_name)
print(json.dumps(app_registration, indent=2))


## 2. Create the service principal id for the application

**Identity Context:** Notebook User

**Credential:** Notebook User's Access Token for Microsoft Graph

This code in this cell creates a service principal for the Blueprint Creator App. The service principal is used to by the application to identify itself to Entra ID when accessing resources associated with the tenant. 

Once created record the following values in the .env file:
1. id property should be written to the ENTRA_ID_BLUEPRINT_CREATOR_SP_OBJECT_ID


In [ ]:
import requests
import json

"""This function calls the Microsoft Graph API to search for a service principal by its app ID"""
def get_service_principal_by_app_id(app_id: str):
    response = requests.get(
        f'https://graph.microsoft.com/v1.0/servicePrincipals?$filter=appId eq \'{app_id}\'',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        }
    )
    if response.status_code != 200:
        print(f"Failed to get service principals: {response.status_code} - {response.text}")
        return None
    # Parse the response and get the value array and return an empty list if not found
    apps = response.json().get('value', [])
    return apps[0] if apps else None

"""This function calls the Microsoft Graph API to create a service principal for the given app ID"""
def create_service_principal(app_id: str):

    # Check to see if the service principal already exists
    service_principal = get_service_principal_by_app_id(app_id)
    if service_principal is not None:
        print(f"Service principal already exists: {service_principal['id']}")
        return service_principal
    body = {
        "appId": app_id
    }
    response = requests.post(
        'https://graph.microsoft.com/v1.0/servicePrincipals',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        },
        json=body
    )
    if response.status_code != 201:
        print(f"Error creating service principal: {response.status_code}: {response.text}")
        return None
    return response.json()

# Get or create the service principal
service_principal = create_service_principal(app_registration['appId'])
print(json.dumps(service_principal, indent=2))

## 3. Grant the delegated permissions required by the Blueprint Creator App's service principal

**Identity Context:** Notebook User

**Credential:** Notebook User's Access Token for Microsoft Graph

This snippet grants the Blueprint Creator App's service principal the permission to impersonate the user for the OAuth scopes required to create and administrator Entra ID Agent Identity Blueprints. The app is granted the permission to impersonate all users for the following required Microsoft Graph permissions:

The [OAuth2 permission scope](https://learn.microsoft.com/en-us/graph/api/oauth2permissiongrant-post?view=graph-rest-1.0&tabs=http) are used to grant the service principal of an application the permission to access a resource on the user's behalf. Permissions can be granted to impersonate a specific user (Principal) or all users (AllPrincipals).



In [ ]:
import requests
import json

"""This function calls the Microsoft Graph API to retrieve the OAuth permissions granted to a service principal"""
def get_oauth_permission_grants(user_token: str, client_id: str):
    response = requests.get(
        url='https://graph.microsoft.com/v1.0/oauth2PermissionGrants',
        headers= {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        },
        params={
            '$filter': f"clientId eq '{client_id}'"
        }
    )

    if response.status_code != 200:
        if response.text:
            print(f"Error getting OAuth permission grants: {response.status_code}: {response.text}")
        else:
            print(f"Error getting OAuth permission grants: {response.status_code}")
    else:
        return response.json()

"""This function calls the Microsoft Graph API to create OAuth permission grants for a service principal"""
def create_oauth_permission_grants(user_token: str, client_id: str, grants: list):
    # Get existing grants
    existing_grants_response = get_oauth_permission_grants(user_token, client_id)
    existing_grants = existing_grants_response.get('value', []) if existing_grants_response else []

    # Iterate over the grants to be created
    for grant in grants:
        consent_type = grant['consentType']
        resource_id = grant['resourceId']
        requested_scopes = grant['scope'].split()
        
        # Check if grant exists for this resource
        grant_exists = False
        for existing_grant in existing_grants:
            if existing_grant['resourceId'] == resource_id:
                existing_scopes = existing_grant.get('scope', '').split()
                scopes_to_add = [s for s in requested_scopes if s not in existing_scopes]
                
                # Skip adding new scopes if they're already there
                if not scopes_to_add:
                    print(f"All requested scopes already exist for resource {resource_id}")
                    grant_exists = True
                # Add the new scopes to the grant
                else:
                    existing_scopes.extend(scopes_to_add)
                    body = {"scope": " ".join(existing_scopes)}
                    response = requests.patch(
                        url=f'https://graph.microsoft.com/v1.0/oauth2PermissionGrants/{existing_grant["id"]}',
                        headers={
                            'Content-Type': 'application/json',
                            'Authorization': f'Bearer {user_token.token}'
                        },
                        json=body
                    )
                    if response.status_code == 204:
                        print(f"Added scopes {scopes_to_add} to existing grant for resource {resource_id}")
                    grant_exists = True
                break
        
        if grant_exists:
            continue
            
        # Create new grant
        body = {
            "clientId": client_id,
            "consentType": consent_type,
            "resourceId": resource_id,
            "scope": grant['scope']
        }

        print(f"Creating new OAuth permission grant: {body}")
        response = requests.post(
            url='https://graph.microsoft.com/v1.0/oauth2PermissionGrants',
            headers={
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {user_token.token}'
            },
            json=body
        )

        if response.status_code != 201:
            if response.text:
                print(f"Error creating OAuth permission grant: {response.status_code}: {response.text}")
        else:
            print("OAuth permission grant created successfully")
    
    # Return the current state of all grants
    return get_oauth_permission_grants(user_token, client_id)

"""This function calls the Microsoft Graph API to delete OAuth permission grants for a service principal"""
def delete_oauth_permission_grant(user_token: str, client_id: str, grant_id: str):
    response = requests.delete(
        url=f'https://graph.microsoft.com/v1.0/oauth2PermissionGrants/{grant_id}',
        headers= {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        }
    )

    if response.status_code != 204:
        if response.text:
            print(f"Error deleting OAuth permission grant: {response.status_code}: {response.text}")
        else:
            print(f"Error deleting OAuth permission grant: {response.status_code}")
    else:
        existing_grants = get_oauth_permission_grants(user_token, client_id)
        print("OAuth permission grant deleted successfully")
        print(existing_grants)

"""This function calls the Microsoft Graph API to get a service principal by its display name"""
def get_service_principal_by_display_name(display_name: str):
    response = requests.get(
        f'https://graph.microsoft.com/v1.0/servicePrincipals?$filter=displayName eq \'{display_name}\'',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        }
    )
    apps = response.json().get('value', [])
    return apps[0] if apps else None

# Load environment variables from .env file
load_dotenv('.env', override=True)

BLUEPRINT_CREATOR_SP_OBJECT_ID = os.getenv("ENTRA_ID_BLUEPRINT_CREATOR_SP_OBJECT_ID")

# Retrieve the Microsoft Graph service principal id which is unique per Entra ID tenant
ms_graph_sp = get_service_principal_by_display_name("Microsoft Graph")['id']

# Establish the required grants
grants = [
    {
        "clientId": BLUEPRINT_CREATOR_SP_OBJECT_ID,
        "consentType": "AllPrincipals",
        "resourceId": ms_graph_sp,
        "scope": "AgentIdentityBlueprint.Create AgentIdentityBlueprint.ReadWrite.All AgentIdentityBlueprintPrincipal.Create AgentIdentityBlueprintPrincipal.ReadWrite.All AgentIdentityBlueprint.AddRemoveCreds.All AgentIdentity.ReadWrite.All"
    }
]

create_oauth_permission_grants(user_token, BLUEPRINT_CREATOR_SP_OBJECT_ID, grants)

## 4. Create a OAuth Public Client URL

**Identity Context:** Notebook User

**Credential:** Notebook User's Access Token for Microsoft Graph

The code in this cell adds a [public client reply URI](https://learn.microsoft.com/en-us/entra/identity-platform/reply-url) to the app registration. Once the user authenticates to Entra the user is redirected to this location. For the purposes of this notebook, the user is redirected to localhost.

In [ ]:
import requests

"""This function calls the Microsoft Graph API to add a public client redirect URI to an app registration"""
def add_public_client_reply_uri(uri: str):
    body = {
        "publicClient": {
            "redirectUris": [
                uri
            ]
        }
    }
    response = requests.patch(
        f'https://graph.microsoft.com/v1.0/applications/{app_registration["id"]}',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        },
        json=body
    )   
    if response.status_code != 204:
        print(f"Error adding public client redirect URI: {response.status_code}: {response.text}")
        return None
    print("Public client redirect URI added successfully")

add_public_client_reply_uri("http://localhost")

# Get updated app registration and dump it
app_registration = get_app_registration_by_display_name(app_display_name)
print(json.dumps(app_registration, indent=2))

# Create Entra ID Agent Blueprint object

## 1. Get an access token for Blueprint Creator App on-behalf-of the user

**Identity Context:** Notebook User

**Credential:** Notebook User's Entra ID Credentials

The code in this cell obtains a delegated access token for the user which will allow the Blueprinter Creator App to impersonate the user to create the required Entra ID Agent Blueprint Identity, permissions, and child objects.

In [ ]:
import os
from dotenv import load_dotenv
from msal import PublicClientApplication

# Load environment variables from .env file
load_dotenv('.env', override=True)

app = PublicClientApplication(
    client_id=os.getenv('ENTRA_ID_BLUEPRINT_CREATOR_APP_CLIENT_ID'),
    authority =f"https://login.microsoftonline.com/{os.getenv('ENTRA_ID_TENANT_ID')}"
)

# Kick off device code flow
flow = app.initiate_device_flow(
 scopes=[
         "https://graph.microsoft.com/AgentIdentityBlueprint.Create",
         "https://graph.microsoft.com/AgentIdentityBlueprint.ReadWrite.All",
         "https://graph.microsoft.com/AgentIdentityBlueprint.AddRemoveCreds.All",
         "https://graph.microsoft.com/AgentIdentityBlueprintPrincipal.Create",
         "https://graph.microsoft.com/AgentIdentityBlueprintPrincipal.ReadWrite.All",
         "https://graph.microsoft.com/AgentIdentity.ReadWrite.All",
         ]
)

# Print the device code message to the console
print(flow['message'])

# Obtain the token once the user has authenticated
result = app.acquire_token_by_device_flow(
    flow)

if 'access_token' in result:
    delegated_token = result['access_token']
    print('Token obtained successfully')
else:
    print(f"Error: {result.get('error_description')}")

print(json.dumps(decode_jwt(delegated_token), indent=2))

## 2. Create the Agent Identity Blueprint object

**Identity Context:** Notebook User

**Credential:** Delegated User Token from Blueprint Creator App

This code in this cell creates the [Entra Agent Identity Blueprint object](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/create-blueprint?tabs=microsoft-graph-api#create-an-agent-identity-blueprint-1). The Entra ID Agent Identity Blueprint acts as the template and parent for Entra ID Agent Identities created underneath it. It holds the credential and can optionally be granted permissions that can be inherited by the Agent Identities it creates.

Once created record the following values in the .env file:
1. appId property should be written to the ENTRA_ID_BLUEPRINT_APP_ID


In [ ]:
import requests
import json
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env', override=True)

"""This function calls the Microsoft Graph API to get an Entra ID Agent Identity Blueprint by its display name"""
def get_agent_blueprint(token, display_name):
    response = requests.get(
        f'https://graph.microsoft.com/beta/applications/microsoft.graph.agentIdentityBlueprint?$filter=startswith(displayName,\'{display_name}\')',
        headers={
            'Content-Type': 'application/json',
            'OData-Version': '4.0',
            'Authorization': f'Bearer {token}'
        }
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }

    if response.status_code != 200:
        if response.text:
            print(f"Error getting agent identity blueprints: {response.status_code}: {response.text}")
        else:
            print(f"Error getting agent identity blueprints: {response.status_code}")
    return result['response'].get('value', [])

"""This function calls the Microsoft Graph API to create an Entra ID Agent Identity Blueprint if one does not already exist with the specified display name"""
def create_agent_identity_blueprint(token, display_name):

    check_for_existing = get_agent_blueprint(token, display_name)

    if check_for_existing == []:

        # Set the sponsor ideas to the information in the .env file
        body = {
            "@odata.type": "Microsoft.Graph.AgentIdentityBlueprint",
            "displayName": display_name,
            "sponsors@odata.bind": [
                f"https://graph.microsoft.com/v1.0/users/{os.getenv('SPONSOR_USER_ID')}",
            ],
            "owners@odata.bind": [
                f"https://graph.microsoft.com/v1.0/users/{os.getenv('SPONSOR_USER_ID')}"
            ]    
        }

        response = requests.post(
            'https://graph.microsoft.com/beta/applications/',
            headers={
                'Content-Type': 'application/json',
                'OData-Version': '4.0',
                'Authorization': f'Bearer {token}'
            },
            json=body,
        )
        
        # Create a result object to hold the status code and response
        result = {
            "status_code": response.status_code,
            "response": response.json() if response.text else {}
        }

        if response.status_code != 201:
            if response.text:
                print(f"Error creating agent identity blueprint: {response.status_code}: {response.text}")
            else:
                print(f"Error creating agent identity blueprint: {response.status_code}")
        else:
            return result['response']
        
    else:
        print(f"Agent Identity Blueprint with display name '{display_name}' already exists.")
        return check_for_existing[0]

# Create the Entra ID Agent Identity Blueprint or return the existing object if it already exists 
agent_blueprint = create_agent_identity_blueprint(delegated_token, "Carl Agent Identity Blueprint 2")
print(json.dumps(agent_blueprint, indent=2))

## 3. Create a service principal for the Agent Identity Blueprint object

**Identity Context:** Notebook User

**Credential:** Delegated User Token from Blueprint Creator App

This code snippet creates a [service princicpal](learn.microsoft.com/en-us/graph/api/resources/agentidentityblueprintprincipal?view=graph-rest-beta) for the Entra ID Agent Identity Blueprint. 



In [ ]:
import requests
import json
from dotenv import load_dotenv

"""This function calls the Microsoft Graph API to get an Entra ID Agent Identity Blueprint Service Principal by its app ID"""
def get_agent_identity_blueprint_prinicipal(agent_blueprint_app_id):
    response = requests.get(
        f'https://graph.microsoft.com/beta/serviceprincipals/microsoft.graph.agentIdentityBlueprintPrincipal?$filter=appId eq \'{agent_blueprint_app_id}\'',
        headers={
            'Content-Type': 'application/json',
            'OData-Version': '4.0',
            'Authorization': f'Bearer {delegated_token}'
        }
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }

    if response.status_code != 200:
        if response.text:
            print(f"Error getting agent identity blueprint principals: {response.status_code}: {response.text}")
        else:
            print(f"Error getting agent identity blueprint principals: {response.status_code}")
    return result['response'].get('value', [])

"""This function calls the Microsoft Graph API to create an Entra ID Agent Identity Blueprint Service Principal if one does not already exist for the specified app ID"""
def create_agent_identity_blueprint_principal(delegated_token, agent_blueprint_app_id):

    check_for_existing = get_agent_identity_blueprint_prinicipal(agent_blueprint_app_id)

    if check_for_existing == []:
        body = {
            "appId": agent_blueprint_app_id
        }

        response = requests.post(
            'https://graph.microsoft.com/beta/serviceprincipals/graph.agentIdentityBlueprintPrincipal',
            headers={
                'Content-Type': 'application/json',
                'OData-Version': '4.0',
                'Authorization': f'Bearer {delegated_token}'
            },
            json=body
        )

        # Create a result object to hold the status code and response
        result = {
            "status_code": response.status_code,
            "response": response.json() if response.text else {}
        }

        if response.status_code != 201:
            if response.text:
                print(f"Error creating service principal: {response.status_code}: {response.text}")
            else:
                print(f"Error creating service principal: {response.status_code}")
        else:
            print("Agent Identity Blueprint Service Principal created successfully")
            return result['response']
    else:
        print(f"Agent Identity Blueprint Principal with app ID '{agent_blueprint_app_id}' already exists.")
        return check_for_existing[0]

# Load environment variables from .env file
load_dotenv('.env', override=True)
agent_blueprint_app_id = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')


blueprint_principal = create_agent_identity_blueprint_principal(delegated_token, agent_blueprint_app_id)
print(json.dumps(blueprint_principal, indent=2))

## 4. Create a client secret for the Entra ID Agent Identity Blueprint

**Identity Context:** Notebook User

**Credential:** Delegated User Token from Blueprint Creator App

The code in this cell creates a [password credential](https://learn.microsoft.com/en-us/graph/api/agentidentityblueprint-addpassword?view=graph-rest-beta) for the Entra ID Agent Identity Blueprint. The credential will be used by the Entra ID Agent Identity Blueprint to create Entra ID Agent Identities and obtain access tokens from Entra ID which are exchanged for access tokens for Entra ID Agent Identities created by the blueprint.

Once created record the following values in the .env file:
1.  secretText property should be written to the ENTRA_ID_BLUEPRINT_CLIENT_SECRET. This property is only available upon creation.

In [ ]:
import requests
import json
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

"""This function calls the Microsoft Graph API to get the Entra ID Agent Identity Blueprint by its app ID"""
def get_agent_blueprint(agent_blueprint_app_id):
    response = requests.get(
        f'https://graph.microsoft.com/beta/applications/{agent_blueprint_app_id}/microsoft.graph.agentIdentityBlueprint',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {delegated_token}'
        }
    )

    if response.status_code != 200:
        if response.text:
            print(f"Error getting service principal credentials: {response.status_code}: {response.text}")
        else:
            print(f"Error getting service principal credentials: {response.status_code}")
    return response.json() if response.text else {}

"""This function calls the Microsoft Graph API to create a password credential for the Entra ID Agent Identity Blueprint"""
def create_blueprint_principal_credential(delegated_token, agent_blueprint_app_id):

    # Check to see if there is already a password credential and if so return the record of it
    check_for_existing = get_agent_blueprint(agent_blueprint_app_id)
    if check_for_existing != {} and check_for_existing.get('passwordCredentials', []) != []:
        print(f"Credential for Entra ID Agent Identity Blueprint for app ID '{agent_blueprint_app_id}' already exists. You need to delete that credential and create a new one.")
        return check_for_existing['passwordCredentials'][0]

    # Create a date one year from now that will be used to expire the credential
    end_date = (datetime.now(timezone.utc) + relativedelta(years=1)).replace(hour=23, minute=59, second=59, microsecond=0)
    formatted_date = end_date.strftime('%Y-%m-%dT%H:%M:%SZ')

    body= {
        "passwordCredential": {
            "displayName": "primary",
            "endDateTime": formatted_date 
        } 
    }

    response = requests.post(
        f'https://graph.microsoft.com/beta/applications/{agent_blueprint_app_id}/addPassword',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {delegated_token}'
        },
        json=body
    )

    if response.status_code != 200:
        if response.text:
            print(f"Error creating credential: {response.status_code}: {response.text}")
        else:
            print(f"Error creating credential: {response.status_code}")
    else:
        print("Entra ID Agent Identity Blueprint principal credential created successfully")
        return response.json()
    
"""This function calls the Microsoft Graph API to delete a password credential for the Entra ID Agent Identity Blueprint"""    
def delete_blueprint_principal_credential(delegated_token, agent_blueprint_app_id):
    check_for_existing = get_agent_blueprint(agent_blueprint_app_id)

    if check_for_existing != {} and check_for_existing.get('passwordCredentials', []) != []:
        credential_id = check_for_existing['passwordCredentials'][0]['keyId']

        response = requests.post(
            f'https://graph.microsoft.com/beta/applications/{agent_blueprint_app_id}/removePassword',
            headers={
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {delegated_token}'
            },
            json={"keyId": credential_id}
        )


        if response.status_code != 204:
            if response.text:
                print(f"Error deleting service principal credential: {response.status_code}: {response.text}")
            else:
                print(f"Error deleting service principal credential: {response.status_code}")
        else:
            print(f"Service principal credential with id '{credential_id}' deleted successfully.")
    else:
        print(f"No service principal credentials found for app ID '{agent_blueprint_app_id}'.")

# Load environment variables from .env file
load_dotenv('.env', override=True)

agent_blueprint_app_id = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')

blueprint_principal_credential = create_blueprint_principal_credential(delegated_token, agent_blueprint_app_id)
#delete_blueprint_principal_credential(delegated_token, agent_blueprint_app_id)
print(json.dumps(blueprint_principal_credential, indent=2))



## 5. Create an identifierUri for the Entra ID Agent Identity Blueprint

**Identity Context:** Notebook User

**Credential:** Delegated User Token from Blueprint Creator App

This code in this cell creates an identifierUri and OAuth scope for the Entra ID Agent Identity Blueprint. In Entra ID, the identifierUri acts as a namespace for OAuth scopes associated with the application. The OAuth scope is a permission in the application.

In [ ]:
import requests
import json
import uuid
from dotenv import load_dotenv

"""This function calls the Microsoft Graph API to get the Entra ID Agent Identity Blueprint by its app ID"""
def get_agent_blueprint(agent_blueprint_app_id):
    response = requests.get(
        f'https://graph.microsoft.com/beta/applications/{agent_blueprint_app_id}/microsoft.graph.agentIdentityBlueprint',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {delegated_token}'
        }
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }

    if response.status_code != 200:
        if response.text:
            print(f"Error getting service principal credentials: {response.status_code}: {response.text}")
        else:
            print(f"Error getting service principal credentials: {response.status_code}")
    return result['response']

"""This function calls the Microsoft Graph API to create an identifier URI for the Entra ID Agent Identity Blueprint"""
def create_identifier_uri(app_id, identifier_uri):
    
    # Pull the agent blueprint
    check_for_existing = get_agent_blueprint(app_id)

    # Grab the existing URIs
    existing_uris = check_for_existing.get('identifierUris', [])

    # Check if the indentifier URI already exists
    if identifier_uri in existing_uris:
        print(f"Identifier URI '{identifier_uri}' already exists for app ID '{app_id}'")
        return check_for_existing
    

    # Generate a unique GUID to uniquely identify the OAuth scope created
    scope_id = str(uuid.uuid4())

    body = {
        "identifierUris": [f"api://{agent_blueprint_app_id}"],
        "api": {
            "oauth2PermissionScopes": [
            {
                "adminConsentDescription": "Allow the application to access the agent on behalf of the signed-in user.",
                "adminConsentDisplayName": "Access agent",
                "id": f"{scope_id}",
                "isEnabled": True,
                "type": "User",
                "value": "access_agent"
            }
            ]
        }
    }

    response = requests.patch(
        f'https://graph.microsoft.com/beta/applications/{agent_blueprint_app_id}',
        headers={
            'Content-Type': 'application/json',
            'OData-Version': '4.0',
            'Authorization': f'Bearer {delegated_token}'
        },
        json=body
    )

    if response.status_code != 204:
        if response.text:
            print(f"Error creating identifier uri: {response.status_code}: {response.text}")
            return None
        else:
            print(f"Error creating identifier uri: {response.status_code}")
            return None
    else:
        print("Identifier URI created successfully")
        new_configuration = get_agent_blueprint(app_id)
        return new_configuration

# Load environment variables from .env file
load_dotenv('.env', override=True)

agent_blueprint_app_id = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')

identifier_uris = create_identifier_uri(agent_blueprint_app_id, f"api://{agent_blueprint_app_id}")
print(json.dumps(identifier_uris, indent=2))

## 6. Add an OAuth2 Permission Grant for the User.Read scope to the Agent Identity Blueprint

**Identity Context:** Notebook User

**Credential:** Notebook User's Access Token for Microsoft Graph

The code in this cell grants permission to the Entra ID Agent Identity Blueprint to impersonate all users within the tenant for the Microsoft Graph User.Read permission.

The permission is granted through the [OAuth2 permission scope](https://learn.microsoft.com/en-us/graph/api/oauth2permissiongrant-post?view=graph-rest-1.0&tabs=http). This grants the service principal of an application the permission to access a resource on the user's behalf. Permissions can be granted to impersonate a specific user (Principal) or all users (AllPrincipals).

In [ ]:
import requests
import json

"""This function calls the Microsoft Graph API to retrieve the OAuth permissions granted to a service principal"""
def get_oauth_permission_grants(user_token: str, client_id: str):
    response = requests.get(
        url='https://graph.microsoft.com/v1.0/oauth2PermissionGrants',
        headers= {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        },
        params={
            '$filter': f"clientId eq '{client_id}'"
        }
    )

    if response.status_code != 200:
        if response.text:
            print(f"Error getting OAuth permission grants: {response.status_code}: {response.text}")
        else:
            print(f"Error getting OAuth permission grants: {response.status_code}")
    else:
        return response.json()

"""This function calls the Microsoft Graph API to create OAuth permission grants for a service principal"""
def create_oauth_permission_grants(user_token: str, client_id: str, grants: list):
    # Get existing grants
    existing_grants_response = get_oauth_permission_grants(user_token, client_id)
    existing_grants = existing_grants_response.get('value', []) if existing_grants_response else []

    # Iterate over the grants to be created
    for grant in grants:
        consent_type = grant['consentType']
        resource_id = grant['resourceId']
        requested_scopes = grant['scope'].split()
        
        # Check if grant exists for this resource
        grant_exists = False
        for existing_grant in existing_grants:
            if existing_grant['resourceId'] == resource_id:
                existing_scopes = existing_grant.get('scope', '').split()
                scopes_to_add = [s for s in requested_scopes if s not in existing_scopes]
                
                # Skip adding new scopes if they're already there
                if not scopes_to_add:
                    print(f"All requested scopes already exist for resource {resource_id}")
                    grant_exists = True
                # Add the new scopes to the grant
                else:
                    existing_scopes.extend(scopes_to_add)
                    body = {"scope": " ".join(existing_scopes)}
                    response = requests.patch(
                        url=f'https://graph.microsoft.com/v1.0/oauth2PermissionGrants/{existing_grant["id"]}',
                        headers={
                            'Content-Type': 'application/json',
                            'Authorization': f'Bearer {user_token.token}'
                        },
                        json=body
                    )
                    if response.status_code == 204:
                        print(f"Added scopes {scopes_to_add} to existing grant for resource {resource_id}")
                    grant_exists = True
                break
        
        if grant_exists:
            continue
            
        # Create new grant
        body = {
            "clientId": client_id,
            "consentType": consent_type,
            "resourceId": resource_id,
            "scope": grant['scope']
        }

        response = requests.post(
            url='https://graph.microsoft.com/v1.0/oauth2PermissionGrants',
            headers={
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {user_token.token}'
            },
            json=body
        )

        if response.status_code != 201:
            if response.text:
                print(f"Error creating OAuth permission grant: {response.status_code}: {response.text}")
        else:
            print("OAuth permission grant created successfully")
    
    # Return the current state of all grants
    return get_oauth_permission_grants(user_token, client_id)
"""This function calls the Microsoft Graph API to delete OAuth permission grants for a service principal"""
def delete_oauth_permission_grant(user_token: str, client_id: str, grant_id: str):
    response = requests.delete(
        url=f'https://graph.microsoft.com/v1.0/oauth2PermissionGrants/{grant_id}',
        headers= {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        }
    )

    if response.status_code != 204:
        if response.text:
            print(f"Error deleting OAuth permission grant: {response.status_code}: {response.text}")
        else:
            print(f"Error deleting OAuth permission grant: {response.status_code}")
    else:
        existing_grants = get_oauth_permission_grants(user_token, client_id)
        print("OAuth permission grant deleted successfully")
        print(existing_grants)

"""This function calls the Microsoft Graph API to get a service principal by its display name"""
def get_service_principal_by_display_name(display_name: str):
    response = requests.get(
        f'https://graph.microsoft.com/v1.0/servicePrincipals?$filter=displayName eq \'{display_name}\'',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {user_token.token}'
        }
    )
    apps = response.json().get('value', [])
    return apps[0] if apps else None

# Load environment variables from .env file
load_dotenv('.env', override=True)

BLUEPRINT_PRINCIPAL_OBJECT_ID = os.getenv("ENTRA_ID_BLUEPRINT_OBJECT_ID")

# Retrieve the Microsoft Graph service principal id which is unique per Entra ID tenant
ms_graph_sp = get_service_principal_by_display_name("Microsoft Graph")['id']

# Establish the required grants
grants = [
    {
        "clientId": BLUEPRINT_PRINCIPAL_OBJECT_ID,
        "consentType": "AllPrincipals",
        "resourceId": ms_graph_sp,
        "scope": "User.Read"
    }
]

create_oauth_permission_grants(user_token, BLUEPRINT_PRINCIPAL_OBJECT_ID, grants)

## 7. Add an inheritable permissions to the Agent Identity Blueprint which will be inherited by its children Agent Identities

**Identity Context:** Notebook User

**Credential:** Delegated User Token from Blueprint Creator App

The code in this cell makes the User.Read permission granted to the Entra ID Agent Identity Blueprint inheritable by Agent Identities it creates.

Entra ID Agent Identity Blueprints can be granted permissions that will be inherited by the child Agent Identities. This concept is called [inherited permissions](https://learn.microsoft.com/en-us/graph/api/agentidentityblueprint-list-inheritablepermissions?view=graph-rest-beta). These can be filtered to all permissions for a given resource (allAllowedScopes), a specific set of set of permissions (enumeratedScopes), or agent identities can be blocked from inheriting any permission from a resource (noScopes).

In [ ]:
import requests

"""This function calls the Microsoft Graph API to add inheritable scopes to an Entra ID Agent Identity Blueprint"""
def add_blueprint_inheritable_scopes(token, blueprint_id, scopes):
    # Get existing inheritable scopes
    existing_permissions = get_blueprint_inheritable_scopes(token, blueprint_id)

    # Create a dictionary of existing permissions by resourceAppId so it can be looked up vs a million for loops
    existing_resources = {
        perm['resourceAppId']: perm 
        for perm in existing_permissions
    }
    for scope in scopes:
        resource_app_id = scope['resourceAppId']
        # Determine if the resourceAppId already exists
        if resource_app_id in existing_resources:
            print("There is an existing scope for this resourceAppId. Checking to see if we need to add more scopes...")
            existing_inheritable_scopes = existing_resources[resource_app_id]['inheritableScopes']['scopes']
            new_scopes = scope['inheritableScopes']['scopes']
            # Determine if there are any new scopes to add
            scopes_to_add = [s for s in new_scopes if s not in existing_inheritable_scopes]
            if scopes_to_add:
                # There are new scopes to add, so combine the existing and new scopes
                combined_scopes = existing_inheritable_scopes + scopes_to_add
                response = requests.patch(
                    f'https://graph.microsoft.com/beta/applications/{blueprint_id}/microsoft.graph.agentIdentityBlueprint/inheritablePermissions/{resource_app_id}',
                    headers={
                        'Content-Type': 'application/json',
                        'OData-Version': '4.0',
                        'Authorization': f'Bearer {token}'
                    },
                    json = {
                        "resourceAppId": resource_app_id,
                        "inheritableScopes": {
                            "@odata.type": "microsoft.graph.enumeratedScopes",
                            "scopes": combined_scopes
                        }
                    }
                )

                if response.status_code != 204:
                    if response.text:
                        print(f"Error updating blueprint scopes: {response.status_code}: {response.text}")
                        return None
                    else:
                        print(f"Error updating blueprint scopes for resourceAppId: {resource_app_id}")
                        return None
                else:
                    print(f"Successfully add the new scopes {scopes_to_add} resourceAppId: {resource_app_id}")
            continue


        response = requests.post(
            f'https://graph.microsoft.com/beta/applications/{blueprint_id}/microsoft.graph.agentIdentityBlueprint/inheritablePermissions',
            headers={
                'Content-Type': 'application/json',
                'OData-Version': '4.0',
                'Authorization': f'Bearer {token}'
            },
            json = scope
        )

        if response.status_code != 201:
            if response.text:
                print(f"Error retrieving blueprint scopes: {response.status_code}: {response.text}")
                return None
            else:
                print(f"Error adding scope: {scope['resourceAppId']}")
                return None

    print(f"Successfully added scope: {scope['resourceAppId']}")
    new_permissions = get_blueprint_inheritable_scopes(token, blueprint_id)
    return new_permissions

"""This function calls the Microsoft Graph API to get inheritable scopes from an Entra ID Agent Identity Blueprint"""
def get_blueprint_inheritable_scopes(token, blueprint_id):
    response = requests.get(
        f'https://graph.microsoft.com/beta/applications/{blueprint_id}/microsoft.graph.agentIdentityBlueprint/inheritablePermissions',
        headers={
            'Content-Type': 'application/json',
            'OData-Version': '4.0',
            'Authorization': f'Bearer {token}'
        }
    )

    if response.status_code != 200:
        if response.text:
            print(f"Error retrieving blueprint scopes: {response.status_code}: {response.text}")
            return None
        else:
            print(f"Error retrieving blueprint scopes: {response.status_code}")
            return None
    else:
        return response.json().get('value', [])

"""This function calls the Microsoft Graph API to remove inheritable scopes from an Entra ID Agent Identity Blueprint"""
def remove_blueprint_inheritable_scopes(token, blueprint_id, scopes):
    for scope in scopes:
        response = requests.delete(
            f'https://graph.microsoft.com/beta/applications/{blueprint_id}/microsoft.graph.agentIdentityBlueprint/inheritablePermissions/{scope["resourceAppId"]}',
            headers={
                'Content-Type': 'application/json',
                'OData-Version': '4.0',
                'Authorization': f'Bearer {token}'
            }
        )

        if response.status_code != 204:
            if response.text:
                print(f"Error removing blueprint scope: {response.status_code}: {response.text}")
                return None
            else:
                print(f"Error removing blueprint scope: {response.status_code}")
                return None
        else:
            print(f"Successfully removed scope: {scope['resourceAppId']}")
            return None

# Load environment variables from .env file
load_dotenv('.env', override=True)

# Scopes make inheritable by Entra ID Agent Identities created under the blueprint
scopes = [
    {
        "resourceAppId": "00000003-0000-0000-c000-000000000000",
        "inheritableScopes": {
            "@odata.type": "microsoft.graph.enumeratedScopes",
            "scopes": [
                "User.Read"
            ]
        }
    }
]

blueprint_app_id = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')
add_blueprint_inheritable_scopes(delegated_token, blueprint_app_id, scopes)
#get_blueprint_inheritable_scopes(delegated_token, blueprint_app_id)
#remove_blueprint_inheritable_scopes(delegated_token, blueprint_app_id, scopes)

## 8. Get an access token for the Microsoft Graph API for the Entra ID Agent Identity Blueprint

**Identity Context:** Agent Identity Blueprint Principal

**Credential:** Agent Identity Blueprint Principal Secret

The code in this cell uses the Entra ID Agent Identity Blueprint credentials to obtain an access token for the Microsoft Graph API. This token will be used to create child Agent Identities. 

In [ ]:
import os
from msal import ConfidentialClientApplication
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env', override=True)

TENANT_ID = os.getenv('ENTRA_ID_TENANT_ID')
ENTRA_ID_BLUEPRINT_CLIENT_ID = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')
ENTRA_ID_BLUEPRINT_CLIENT_SECRET = os.getenv('ENTRA_ID_BLUEPRINT_CLIENT_SECRET')

# Obtain access token
app = ConfidentialClientApplication(
    client_id=ENTRA_ID_BLUEPRINT_CLIENT_ID,
    client_credential=ENTRA_ID_BLUEPRINT_CLIENT_SECRET,
    authority=f"https://login.microsoftonline.com/{TENANT_ID}",
)

blueprint_token = app.acquire_token_for_client(
    scopes=["https://graph.microsoft.com/.default"]
)['access_token']

if blueprint_token:
    print('Token obtained successfully')

# DEMO 1: Demonstrate an autonomous agent accessing the Microsoft Graph API



## Part 1: Create an Entra ID Agent Identity that will act as an autonomous agent

**Identity Context:** Agent Identity Blueprint Principal

**Credential:** Agent Identity Blueprint Principal Secret

The code in this cell creates an [Entra ID Agent Identity](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/create-delete-agent-identities?tabs=microsoft-graph-api) which is the child of the Entra ID Agent Identity Blueprint.

Once created record the following values in the .env file:
1.  The id property should be written to the ENTRA_ID_AGENT_IDENTITY_AUTON_OBJECT_ID.

In [ ]:
import requests
import json
from dotenv import load_dotenv

"""This function calls the Microsoft Graph API to get Entra ID Agent Identities associated with a specific blueprint"""
def get_agent_identities(token, agent_blueprint_app_id):
    response = requests.get(
        url='https://graph.microsoft.com/beta/servicePrincipals/microsoft.graph.agentIdentity',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {token}'
        },
        params={
            '$filter': f'agentIdentityBlueprintId eq \'{agent_blueprint_app_id}\''
        }   
    )

    if response.status_code != 200:
        if response.text:
            print(f"Error getting agent identities: {response.status_code}: {response.text}")
        else:
            print(f"Error getting agent identities: {response.status_code}")
    else:
        return response.json()

"""This function calls the Microsoft Graph API to create an Entra ID Agent Identity associated with a specific blueprint"""
def create_agent_identity(agent_blueprint_app_id, delegated_token, blueprint_token, display_name):
    # Get a list of all agents associated with the blueprint. This prevents us from creating agents with duplicate display names which Entra ID will allow but seems stupid
    agent_identities = get_agent_identities(delegated_token,agent_blueprint_app_id)
    
    # Do a lazy evaluation to search the existing agents to determine if one with the specified display name already exists
    existing_agent = next((agent for agent in agent_identities.get('value', []) if agent['displayName'] == display_name), None)
    
    # If you find an existing agent, return a message to the user and the existing agent object
    if existing_agent:
        print(f"Agent Identity with display name '{display_name}' already exists.")
        return existing_agent
    
    # If no existing agent found, create a new one
    body = {
        "displayName": display_name,
        "agentIdentityBlueprintId": agent_blueprint_app_id,
        "sponsors@odata.bind": [
            f"https://graph.microsoft.com/v1.0/users/{os.getenv('SPONSOR_USER_ID')}",
            f"https://graph.microsoft.com/v1.0/groups/{os.getenv('SPONSOR_GROUP_ID')}"
        ]
    }

    response = requests.post(
        f'https://graph.microsoft.com/beta/serviceprincipals/Microsoft.Graph.AgentIdentity',
        headers={
            'Content-Type': 'application/json',
            'OData-Version': '4.0',
            'Authorization': f'Bearer {blueprint_token}'
        },
        json=body
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }

    if response.status_code != 201:
        if response.text:
            print(f"Error creating agent service principal: {response.status_code}: {response.text}")
        else:
            print(f"Error creating agent service principal: {response.status_code}")
    else:
        return result['response']

# Load environment variables from .env file
load_dotenv('.env')
agent_blueprint_app_id = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')

agent_identity = create_agent_identity(agent_blueprint_app_id, delegated_token,blueprint_token, "My agent 1")
print(json.dumps(agent_identity, indent=2))



## Part 2: Add App Role to an Agent Identity

**Identity Context:** Notebook User

**Credential:** Delegated User Token from Blueprint Creator App

This code in this cell adds the User.Read.All permission (creation of App Role Assignment in Entra ID world) to the Entra ID Agent Identity. Note that the Microsoft Graph application has a different object id in every Entra ID tenant so you'll need to pull your own. An app role is a direct permission to an application's service principal allowing it to exercise those permissions directly.

In [ ]:
import requests
from dotenv import load_dotenv

"""This function calls the Microsoft Graph API to get the app roles assigned to an Entra ID Agent Identity"""
def get_app_roles(agent_object_id, token):
    response = requests.get(
        f'https://graph.microsoft.com/beta/servicePrincipals/{agent_object_id}/appRoleAssignments',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {token}'
        }
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }

    if result["status_code"] != 200:
        print(f"Error getting app roles: {result['status_code']}: {result['response']}")
    else:
        return(result['response'].get('value', []))

"""This function calls the Microsoft Graph API to assign an app role to an Entra ID Agent Identity"""
def add_app_role(app_role_id, resource_id, agent_object_id, token):
    current_roles = get_app_roles(agent_object_id, token)

    # Check to see if role is already assigned
    for role in current_roles:
        if role['appRoleId'] == app_role_id and role['resourceId'] == resource_id:
            print("App role already assigned")
            return role

    body = {
        "principalId": agent_object_id,
        "resourceId": resource_id,
        "appRoleId": app_role_id
    }

    response = requests.post(
        f'https://graph.microsoft.com/beta/servicePrincipals/{agent_object_id}/appRoleAssignments',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {token}'
        },
        json=body
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }

    if result['status_code'] != 201:
        if result['response']:
            print(f"Error assigning app role: {result['status_code']}: {result['response']}")
        else:
            print(f"Error assigning app role: {result['status_code']}")
    else:
        print("App role assigned successfully")
        return result['response']
    
"""This function calls the Microsoft Graph API to remove an app role from an Entra ID Agent Identity"""
def remove_app_role(app_role_assignment_id, agent_object_id, token):
    response = requests.delete(
        f'https://graph.microsoft.com/beta/servicePrincipals/{agent_object_id}/appRoleAssignments/{app_role_assignment_id}',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {token}'
        }
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }

    if result["status_code"] != 204:
        print(f"Error removing app role: {result['status_code']}: {result['response']}")
    else:
        print("App role removed successfully")
        return(result)
    
# Load environment variables from .env file
load_dotenv('.env', override=True)

ENTRA_ID_BLUEPRINT_OBJECT_ID = os.getenv('ENTRA_ID_BLUEPRINT_OBJECT_ID')
ENTRA_ID_AGENT_IDENTITY_OBJECT_ID = os.getenv('ENTRA_ID_AGENT_IDENTITY_AUTON_OBJECT_ID')

add_app_role(
    # Microsoft Graph: User.Read.All
    app_role_id="df021288-bdef-4463-88db-98f22de89214",
    # Microsoft Graph API Service Principal object id; this is per Entra ID tenant
    resource_id='bd0fb293-be5c-44d2-9b95-db0d941be021',
    agent_object_id=ENTRA_ID_AGENT_IDENTITY_OBJECT_ID,
    token=delegated_token
)

#get_app_roles(
#    agent_object_id=ENTRA_ID_AGENT_IDENTITY_OBJECT_ID,
#    token=user_token.token
#)

#remove_app_role(
#    app_role_assignment_id='WAwN8p5Lv0Swm4U17dRehlnBkTw40rFGr2INQOAOYJY',
#    agent_object_id=ENTRA_ID_AGENT_IDENTITY_OBJECT_ID,
#    token=user_token.token
#)


## Part 3: Obtain an exchange token for the Entra ID Agent Identity Blueprint

**Identity Context:** Agent Identity Blueprint Principal

**Credential:** Agent Identity Blueprint Principal Secret

The code in this cell obtains an access token for the Agent Identity Blueprint that can be exchanged for an access token for its Agent Identities.

In [ ]:
import requests
import json
import base64
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env', override=True)

tenant_id = os.getenv('ENTRA_ID_TENANT_ID')
agent_blueprint_client_id = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')
agent_blueprint_client_secret = os.getenv('ENTRA_ID_BLUEPRINT_CLIENT_SECRET')
agent_identity_client_id=os.getenv('ENTRA_ID_AGENT_IDENTITY_AUTON_OBJECT_ID')

"""This function calls the Microsoft identity platform to get an exchange token for an Entra ID Agent Identity Blueprint"""
def get_blueprint_assertion(agent_blueprint_client_id, agent_blueprint_client_secret, tenant_id, agent_identity_client_id):
    body = {
        'client_id': agent_blueprint_client_id,
        'client_secret': agent_blueprint_client_secret,
        'grant_type': 'client_credentials',
        'scope': 'api://AzureADTokenExchange/.default',
        'fmi_path': agent_identity_client_id
    }

    response = requests.post(
        url=f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token',
        headers={
            'Content-Type': 'application/x-www-form-urlencoded'
        },
        data=body
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }


    if response.status_code != 200:
        if result['response']:
            print(f"Error obtaining blueprint assertion: {result['status_code']}: {result['response']}")
        else:
            print(f"Error obtaining blueprint assertion: {result['status_code']}")
    else:
        return result['response']['access_token']
    
blueprint_assertion = get_blueprint_assertion(
    agent_blueprint_client_id, 
    agent_blueprint_client_secret, 
    tenant_id, 
    agent_identity_client_id
)

print(json.dumps(decode_jwt(blueprint_assertion), indent=2))




## Part 4: Obtain an access token for the Agent Identity

The code in this cell exchanges the Agent Identity Blueprint access token for an access token for the Agent Identity for the Microsoft Graph API scope.

**Identity Context:** Agent Identity Blueprint Principal

**Credential:** Exchange token for Agent Identity Blueprint

In [ ]:
import requests
import json
from dotenv import load_dotenv

"""This function calls Entra ID to get an access token for the agent identity using the blueprint assertion"""
def get_agent_access_token(agent_id, blueprint_assertion):
    body = {
        'client_id': agent_id,
        'scope': 'https://graph.microsoft.com/.default',
        'grant_type': 'client_credentials',
        'client_assertion_type': 'urn:ietf:params:oauth:client-assertion-type:jwt-bearer',
        'client_assertion': blueprint_assertion
    }

    response = requests.post(
        url=f'https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token',
        headers={
            'Content-Type': 'application/x-www-form-urlencoded'
        },
        data=body
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}    
    }
    
    if result['status_code'] != 200:
        if result['response']:
            print(f"Error obtaining agent access token: {result['status_code']}: {result['response']}")
        else:
            print(f"Error obtaining agent access token: {result['status_code']}")
    else:
        return result['response']['access_token']
    
# Load environment variables from .env file
load_dotenv('.env', override=True)

agent_id = os.getenv('ENTRA_ID_AGENT_IDENTITY_AUTON_OBJECT_ID')
    
agent_access_token = get_agent_access_token(agent_id, blueprint_assertion)
print(json.dumps(decode_jwt(agent_access_token), indent=2))

## Part 5: Query the Microsoft Graph API using the Agent Identity (Autonomous)

The code in this cell uses the Agent Identity access token to make a call to the Microsoft Graph API.

**Identity Context:** Agent Identity

**Credential:** Access token for Agent Identity

In [ ]:
import requests
import json
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env', override=True)

user_object_id = os.getenv('SPONSOR_USER_ID')

# Obtain an access token for the agent identity for the Microsoft Graph API
def query_user_info(user_object_id):
    response = requests.get(
        url=f'https://graph.microsoft.com/v1.0/users/{user_object_id}',
        headers={
            'Authorization': f'Bearer {agent_access_token}'
        }
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}    
    }
    
    if result['status_code'] != 200:
        if result['response']:
            print(f"Error obtaining user info: {result['status_code']}: {result['response']}")
        else:
            print(f"Error obtaining user info: {result['status_code']}")
    else:
        return result['response']

user_info = query_user_info(user_object_id)
print(json.dumps(user_info, indent=2))


# DEMO 2: Demonstrate on-behalf-of agent



## Part 1: Create an Entra ID Agent Identity that will act as an obo agent

**Authentication** Entra ID Agent Identity Blueprint principal access token

This creates an Entra ID Agent Identity which is the child of the Entra ID Agent Identity Blueprint. The appId property should be recorded and written to the .env file into the variable ENTRA_ID_AGENT_IDENTITY_OBJECT_ID.

In [ ]:
import requests
import json
from dotenv import load_dotenv

"""This function calls the Microsoft Graph API to get Entra ID Agent Identities associated with a specific blueprint"""
def get_agent_identities(agent_blueprint_app_id):
    response = requests.get(
        url='https://graph.microsoft.com/beta/servicePrincipals/microsoft.graph.agentIdentity',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {delegated_token}'
        },
        params={
            '$filter': f'agentIdentityBlueprintId eq \'{agent_blueprint_app_id}\''
        }   
    )

    # Create a result object to hold the status code and response
    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }
    if response.status_code != 200:
        if response.text:
            print(f"Error getting agent identities: {response.status_code}: {response.text}")
        else:
            print(f"Error getting agent identities: {response.status_code}")
    else:
        return result['response']

"""This function calls the Microsoft Graph API to create an Entra ID Agent Identity associated with a specific blueprint"""
def create_agent_identity(agent_blueprint_app_id, blueprint_token, display_name):
    # Get a list of all agents associated with the blueprint. This prevents us from creating agents with duplicate display names which Entra ID will allow but seems stupid
    agent_identities = get_agent_identities(agent_blueprint_app_id)
    
    # Do a lazy evaluation to search the existing agents to determine if one with the specified display name already exists
    existing_agent = next((agent for agent in agent_identities.get('value', []) if agent['displayName'] == display_name), None)
    
    # If you find an existing agent, return a message to the user and the existing agent object
    if existing_agent:
        print(f"Agent Identity with display name '{display_name}' already exists.")
        return existing_agent
    
    # If no existing agent found, create a new one
    body = {
        "displayName": display_name,
        "agentIdentityBlueprintId": agent_blueprint_app_id,
        "sponsors@odata.bind": [
            f"https://graph.microsoft.com/v1.0/users/{os.getenv('SPONSOR_USER_ID')}",
            f"https://graph.microsoft.com/v1.0/groups/{os.getenv('SPONSOR_GROUP_ID')}"
        ]
    }

    response = requests.post(
        f'https://graph.microsoft.com/beta/serviceprincipals/Microsoft.Graph.AgentIdentity',
        headers={
            'Content-Type': 'application/json',
            'OData-Version': '4.0',
            'Authorization': f'Bearer {blueprint_token}'
        },
        json=body
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }

    if response.status_code != 201:
        if response.text:
            print(f"Error creating agent service principal: {response.status_code}: {response.text}")
        else:
            print(f"Error creating agent service principal: {response.status_code}")
    else:
        print("Agent identity created successfully")
        return result['response']

# Load environment variables from .env file
load_dotenv('.env')
agent_blueprint_app_id = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')

agent_identity = create_agent_identity(agent_blueprint_app_id, blueprint_token, "My OBO Agent 1")
print(json.dumps(agent_identity, indent=2))



## Part 2: Get an exchange token for the Entra ID Agent Identity Blueprint

**Identity Context:** Agent Identity Blueprint Principal

**Credential:** Agent Identity Blueprint Principal Secret

The code in this cell obtains an access token for the Agent Identity Blueprint that can be exchanged for an access token for its Agent Identities.

In [ ]:
import requests
import json
import base64
from dotenv import load_dotenv

"""This function calls the Microsoft identity platform to get an exchange token for an Entra ID Agent Identity Blueprint"""
def get_blueprint_assertion(agent_blueprint_client_id, agent_blueprint_client_secret, tenant_id, agent_identity_client_id):
    body = {
        'client_id': agent_blueprint_client_id,
        'client_secret': agent_blueprint_client_secret,
        'grant_type': 'client_credentials',
        'scope': 'api://AzureADTokenExchange/.default',
        'fmi_path': agent_identity_client_id
    }

    response = requests.post(
        url=f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token',
        headers={
            'Content-Type': 'application/x-www-form-urlencoded'
        },
        data=body
    )

    result = {
        "status_code": response.status_code,
        "response": response.json() if response.text else {}
    }


    if response.status_code != 200:
        if result['response']:
            print(f"Error obtaining blueprint assertion: {result['status_code']}: {result['response']}")
        else:
            print(f"Error obtaining blueprint assertion: {result['status_code']}")
    else:
        return result['response']['access_token']
    
# Load environment variables from .env file
load_dotenv('.env', override=True)

tenant_id = os.getenv('ENTRA_ID_TENANT_ID')
agent_blueprint_client_id = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')
agent_blueprint_client_secret = os.getenv('ENTRA_ID_BLUEPRINT_CLIENT_SECRET')
agent_identity_client_id=os.getenv('ENTRA_ID_AGENT_IDENTITY_OBO_OBJECT_ID')

blueprint_exchange_token = get_blueprint_assertion(
    agent_blueprint_client_id, 
    agent_blueprint_client_secret, 
    tenant_id, 
    agent_identity_client_id
)

print(json.dumps(decode_jwt(blueprint_exchange_token), indent=2))




## Part 3: Get an access token for the Entra ID Agent Identity Blueprint for the user

**Identity Context:** Notebook User

**Credential:** Notebook User credentials

The code in this cell obtains an access token for the user for access to the Entra ID Agent Identity Blueprint. This access token will be used for the on-behalf-of flow demonstrated in later cells.

In [ ]:
import os
from msal import ConfidentialClientApplication
from dotenv import load_dotenv
import webbrowser

# Load environment variables from .env file
load_dotenv('.env', override=True)

TENANT_ID = os.getenv('ENTRA_ID_TENANT_ID')
CLIENT_ID = os.getenv('ENTRA_ID_SERVICE_PRINCIPAL_CLIENT_ID')
CLIENT_SECRET = os.getenv('ENTRA_ID_SERVICE_PRINCIPAL_SECRET')
BLUEPRINT_APP_ID = os.getenv('ENTRA_ID_BLUEPRINT_APP_ID')
REDIRECT_URI = "http://localhost:8000"

app = ConfidentialClientApplication(
    client_id=CLIENT_ID,
    client_credential=CLIENT_SECRET,
    authority=f"https://login.microsoftonline.com/{TENANT_ID}"
)

# Get the authorization URL - user needs to visit this and authenticate
auth_url = app.get_authorization_request_url(
    scopes=[f"api://{BLUEPRINT_APP_ID}/access_agent"],
    redirect_uri=REDIRECT_URI
)

print(f"Visit this URL to authenticate:\n{auth_url}\n")
webbrowser.open(auth_url)

# After user authenticates, they'll be redirected to http://localhost:8000?code=...
# Extract the 'code' parameter from that URL
auth_code = input("Paste the 'code' parameter from the redirect URL: ")

# Exchange the authorization code for a token
result = app.acquire_token_by_authorization_code(
    code=auth_code,
    scopes=[
        f"api://{BLUEPRINT_APP_ID}/access_agent"],
    redirect_uri=REDIRECT_URI
)

if 'access_token' in result:
    incoming_user_token = result['access_token']
    print('User token obtained successfully')
    print(json.dumps(decode_jwt(incoming_user_token), indent=2))
else:
    print(f"Error: {result.get('error_description')}")

## Part 4: Obtain an access token on behalf of the user for the agent identity

**Identity Context:** Notebook User

**Credential:** Access token for Notebook User

The code in this cell obtains an access token for the user for access to the Entra ID Agent Identity Blueprint. This access token will be used for the on-behalf-of flow demonstrated in later cells.

In [ ]:
import requests
import json
from dotenv import load_dotenv

"""This function calls Entra ID to get an access token for the agent identity using the blueprint exchange token and incoming user token"""
def get_agent_access_token(agent_id, blueprint_exchange_token, user_token):
    body = {
        'client_id': agent_id,
        'scope': 'User.Read',
        'client_assertion_type': 'urn:ietf:params:oauth:client-assertion-type:jwt-bearer',
        'client_assertion': blueprint_exchange_token,
        'grant_type': 'urn:ietf:params:oauth:grant-type:jwt-bearer',
        'assertion': user_token,
        'requested_token_use': 'on_behalf_of'
    }

    response = requests.post(
        url=f'https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token',
        headers={
            'Content-Type': 'application/x-www-form-urlencoded'
        },
        data=body
    )
    
    if response.status_code != 200:
        if response.text:
            print(f"Error obtaining agent access token: {response.status_code}: {response.text}")
        else:
            print(f"Error obtaining agent access token: {response.status_code}")
    else:
        return response.json().get('access_token')
    

# Load environment variables from .env file
load_dotenv('.env', override=True)

agent_id = os.getenv('ENTRA_ID_AGENT_IDENTITY_OBO_OBJECT_ID')
print(agent_id)

    
agent_obo_access_token = get_agent_access_token(agent_id, blueprint_exchange_token, incoming_user_token)
print(json.dumps(decode_jwt(agent_obo_access_token), indent=2))

## Part 5: Query the Graph API for the user's profile on behalf of the user

**Identity Context:** Notebook User

**Credential:** Access token for Agent Identity OBO Notebook User

The code in this cell calls the Microsoft Graph API on-behalf-of the user to obtain the user's profile information.

In [ ]:
import requests
from dotenv import load_dotenv

"""This function calls the Microsoft Graph API to get user data using the agent OBO access token"""
def get_user_data(user_id):
    body = {}

    response = requests.get(
        url=f'https://graph.microsoft.com/v1.0/users/{user_id}',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {agent_obo_access_token}'
        }
    )
    return response.json()

print(json.dumps(get_user_data(os.getenv('SPONSOR_USER_ID')), indent=2))

# Other shit still being worked on

## Delete Entra ID Agent Identities

Before deleting an Entra ID Agent Identity Blueprint, the agent identities that were created as children of that blueprint should be disabled. After an appropriate period of time, those agent identities should be deleted. Once all agent identities are deleted, the blueprint can be deleted.

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env', override=True)

blueprint_id = os.getenv('7ef53373-eaf7-444e-b7d8-4eab163f82ec')

def get_agent_blueprints():
    response = requests.get(
        url='https://graph.microsoft.com/beta/applications/microsoft.graph.agentIdentityBlueprint',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {delegated_token}'
        }
    )
    return response.json()

def get_agent_identities(blueprint_id):
    response = requests.get(
        url='https://graph.microsoft.com/beta/servicePrincipals/microsoft.graph.agentIdentity',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {delegated_token}'
        },
        params={
            '$filter': f'agentIdentityBlueprintId eq \'{blueprint_id}\''
        }   
    )
    return response.json()


def disable_agent_identity(agent_id):
    body = {
        "accountEnabled": False
    }
    response = requests.patch(

        url=f'https://graph.microsoft.com/beta/servicePrincipals/{agent_id}',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {delegated_token}'
        },
        json=body
    )
    if response.status_code == 204:
        print(f"Successfully disabled agent identity: {agent_id}")
        return response.status_code
    else:
        print(f"Failed to disable agent identity: {agent_id}. Status code: {response.status_code}. Error: {response.text}")
        return response.status_code

def delete_agent_identity(agent_id):
    response = requests.delete(
        url=f'https://graph.microsoft.com/beta/servicePrincipals/{agent_id}',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {delegated_token}'
        }
    )
    if response.status_code == 204:
        print(f"Successfully deleted agent identity: {agent_id}")
        return response.status_code
    else:
        print(f"Failed to delete agent identity: {agent_id}. Status code: {response.status_code}. Error: {response.text}")
        return response.status_code
    
def delete_agent_blueprint(blueprint_id):
    response = requests.delete(
        url=f'https://graph.microsoft.com/beta/applications/{blueprint_id}',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {delegated_token}'
        }
    )
    if response.status_code == 204:
        print(f"Successfully deleted agent identity blueprint: {blueprint_id}")
        return response.status_code
    else:
        print(f"Failed to delete agent identity blueprint: {blueprint_id}. Status code: {response.status_code}. Error: {response.text}")
        return response.status_code

get_agent_blueprints()
# Agent Identity Cleanup
#agent_identities = get_agent_identities(blueprint_id)
#for agent in agent_identities.get('value', []):
#    print(f"Disabling agent identity: {agent['id']}")
#    disable_agent_identity(agent['id'])
#    delete_agent_identity(agent['id'])

# Blueprint Cleanup    
#delete_agent_blueprint(blueprint_id)
